Underlying Principle
--------------------

The principle is simple: we define two distances, one for the content
($D_C$) and one for the style ($D_S$). $D_C$ measures how different the content
is between two images while $D_S$ measures how different the style is
between two images. Then, we take a third image, the input, and
transform it to minimize both its content-distance with the
content-image and its style-distance with the style-image. Now we can
import the necessary packages and begin the neural transfer.

Importing Packages and Selecting a Device
-----------------------------------------
Below is a  list of the packages needed to implement the neural transfer.

-  ``torch``, ``torch.nn``, ``numpy`` (indispensables packages for
   neural networks with PyTorch)
-  ``torch.optim`` (efficient gradient descents)
-  ``PIL``, ``PIL.Image``, ``matplotlib.pyplot`` (load and display
   images)
-  ``torchvision.transforms`` (transform PIL images into tensors)
-  ``torchvision.models`` (train or load pre-trained models)
-  ``copy`` (to deep copy the models; system package)



In [0]:
%matplotlib inline


Neural Transfer Using PyTorch
=============================


**Author**: `Alexis Jacq <https://alexis-jacq.github.io>`_
 
**Edited by**: `Winston Herring <https://github.com/winston6>`_

Introduction
------------

This tutorial explains how to implement the `Neural-Style algorithm <https://arxiv.org/abs/1508.06576>`__
developed by Leon A. Gatys, Alexander S. Ecker and Matthias Bethge.
Neural-Style, or Neural-Transfer, allows you to take an image and
reproduce it with a new artistic style. The algorithm takes three images,
an input image, a content-image, and a style-image, and changes the input 
to resemble the content of the content-image and the artistic style of the style-image.

 
.. figure:: /_static/img/neural-style/neuralstyle.png
   :alt: content1



In [0]:
from __future__ import print_function #future문 : 호환성 있는 코드를 작성하기 위해 사용 / print_function : 괄호 한의 인자 재지정 및 출력 분리 --> future 모듈로부터 print_function을 불러내는 것

import torch #pytorch를 사용하기 위해 torch 모듈 import
import torch.nn as nn #nn(neural network)을 구현하기 위한 모듈(torch.nn)을 nn이라는 이름으로 불러오기
import torch.nn.functional as F #neural network(nn)을 구현하기 위한 모듈(torch.nn.functional)을 F라는 이름으로 불러오기
import torch.optim as optim #경사하강법을 사용하기 위한 모듈(torch.optim)을 optim이라는 이름으로 불러오기

from PIL import Image #이미지를 사용하기 위한 PIL모듈 사용
import matplotlib.pyplot as plt #그래프를 사용하기(그리기) 위한 matplotlib.pyplot 모듈을 plt라는 이름으로 불러오기

import torchvision.transforms as transforms #PIL이미지를 텐서로 변환하기 위한 torchvision.transforms 모듈을 transforms라는 이름으로 불러오기
import torchvision.models as models # 사전에 학습된 모델들을 학습하거나 읽는 torchvision.models라는 모듈을 models라는 이름으로 불러오기

import copy #모델이나 시스템 패키지를 복사할 수 있는 copy모듈을 불러오기

Next, we need to choose which device to run the network on and import the
content and style images. Running the neural transfer algorithm on large
images takes longer and will go much faster when running on a GPU. We can
use ``torch.cuda.is_available()`` to detect if there is a GPU available.
Next, we set the ``torch.device`` for use throughout the tutorial. Also the ``.to(device)``
method is used to move tensors or modules to a desired device. 



In [0]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") #그래픽 처리를 GPU가 아니라 파이썬(cuda 환경을 사용해서)에서도 할 수 있게끔 처리 / .cuda() 명령어를 통해 수행 가능(.cpu()를 통해 CPU로 되돌릴 수 있음)

Loading the Images
------------------

Now we will import the style and content images. The original PIL images have values between 0 and 255, but when
transformed into torch tensors, their values are converted to be between
0 and 1. The images also need to be resized to have the same dimensions.
An important detail to note is that neural networks from the
torch library are trained with tensor values ranging from 0 to 1. If you
try to feed the networks with 0 to 255 tensor images, then the activated
feature maps will be unable sense the intended content and style.
However, pre-trained networks from the Caffe library are trained with 0
to 255 tensor images. 


.. Note::
    Here are links to download the images required to run the tutorial:
    `picasso.jpg <https://pytorch.org/tutorials/_static/img/neural-style/picasso.jpg>`__ and
    `dancing.jpg <https://pytorch.org/tutorials/_static/img/neural-style/dancing.jpg>`__.
    Download these two images and add them to a directory
    with name ``images`` in your current working directory.



In [0]:
#출력 이미지와 크기 설정(이미지 사이즈 변수 :imsize)
imsize = 512 if torch.cuda.is_available() else 128  #사이즈 = 512, 쿠다 환경을 사용할 수 없다면 128사이즈로(작은 사이즈)

#loader 데이터셋에 transforms에 넣을 함수들의 형식을 설정해서 저장(Data augmentaion을 위한 --> 크기가 제각각인 이미지들을 nn에 적용시키기 위해)
loader = transforms.Compose([ #transforms에 넣을 함수들을 직렬로 나열(보기 편하게 하려고)
    transforms.Resize(imsize),  #이 cnn에서 사용할 이미지들의 사이즈를 imsize(512 or 128)로 설정
    transforms.ToTensor()])  #torch tensor로의 변환

#이미지 이름(이미지 저장 주소)을 인자로 받는 image_loader함수 생성 
def image_loader(image_name):
    image = Image.open(image_name) #특정 이미지 불러오기
    image = loader(image).unsqueeze(0) #네트워크의 입력 차원을 맞추기 위해 필요한, loader에 저장한 이미지의 가짜 배치 차원을 생성 #차원이 다른 이미지
    return image.to(device, torch.float) #현재 프로그램을 돌리는 장치로 torch.float(자료형임)형으로 처리한 이미지 반환

#이미지 불러온 후 image_loader 함수를 사용해 nn을 돌릴 수 있도록 형식 설정
style_img = image_loader("./data/images/neural-style/picasso.jpg") 
content_img = image_loader("./data/images/neural-style/dancing.jpg")

assert style_img.size() == content_img.size(), \ #스타일 이미지와 내용 이미지의 사이즈가  512나 128로 동일하지 않을 경우 아래의 에러메세지 출력
    "we need to import style and content images of the same size"

Now, let's create a function that displays an image by reconverting a 
copy of it to PIL format and displaying the copy using 
``plt.imshow``. We will try displaying the content and style images 
to ensure they were imported correctly.



In [0]:
unloader = transforms.ToPILImage()  #PIL이미지로 재 변환

plt.ion() #이미지 갱신

#이미지를 표시하기 위한 함수 imshow 생성
def imshow(tensor, title=None):
    image = tensor.cpu().clone() #이미지에 변화를 가했을 때 텐서의 값에 변화가 적용되지 않도록 사본 생성
    image = image.squeeze(0) #앞서 설정한 가짜 배치 차원 제거
    image = unloader(image) #이미지를 PIL이미지로 변환
    plt.imshow(image) #이미지 띄워주기(보여주기)
    if title is not None: #만약 이미지의 타이틀에 뭔가 다른 이름이 있다면
        plt.title(title) #인자로 받을 타이틀 이름으로 변경 <<?
    plt.pause(0.001) #그리는 부분이 갱신될 수 있도록 잠시 정지(0.001초 동안)


plt.figure() #이미지 보여주기
imshow(style_img, title='Style Image') #앞서 불러온 스타일 이미지와 이미지의 타이틀을 imshow함수에 넣기

plt.figure()
imshow(content_img, title='Content Image') #앞서 불러온 내용 이미지와 이미지의 타이틀을 imshow함수에 넣기

Loss Functions
--------------
Content Loss
~~~~~~~~~~~~

The content loss is a function that represents a weighted version of the
content distance for an individual layer. The function takes the feature
maps $F_{XL}$ of a layer $L$ in a network processing input $X$ and returns the
weighted content distance $w_{CL}.D_C^L(X,C)$ between the image $X$ and the
content image $C$. The feature maps of the content image($F_{CL}$) must be
known by the function in order to calculate the content distance. We
implement this function as a torch module with a constructor that takes
$F_{CL}$ as an input. The distance $\|F_{XL} - F_{CL}\|^2$ is the mean square error
between the two sets of feature maps, and can be computed using ``nn.MSELoss``.

We will add this content loss module directly after the convolution
layer(s) that are being used to compute the content distance. This way
each time the network is fed an input image the content losses will be
computed at the desired layers and because of auto grad, all the
gradients will be computed. Now, in order to make the content loss layer
transparent we must define a ``forward`` method that computes the content
loss and then returns the layer’s input. The computed loss is saved as a
parameter of the module.




In [0]:
#콘텐츠 로스 모듈 설정
#콘텐츠 로스 : 원본 이미지(사전에 학습했던 --> 아래 코드에서는 target으로 정의)와 입력 이미지를 CNN에 통과시킨 다음, 같은 레이어의 출력값들과의 차를 구한 다음 그 차의 제곱합으로 정의한 것 = 최종적으로 이 차이가 작은 것을 찾는 것이 목표

class ContentLoss(nn.Module): 

    def __init__(self, target,):
        super(ContentLoss, self).__init__() #super() : 부모 클래스에서 정의된 메소드를 상속받고 싶을 때 사용 / 대상 콘텐츠를 분리하면서 이 손실값이 명시된 값이라는 것을 나타냄
        self.target = target.detach() #target의 텐서를 더이상 추적하지 않고(이후에 발생하는 계산을 기록하지 않음) --> 역전파 : 이미 한번 계산하고 선택했던 거는 다시 계산하지 않기 위함

    def forward(self, input):
        self.loss = F.mse_loss(input, self.target) #입력 이미지와 타겟(이미 학습된 이미지) 간의 로스 계산
        return input #입력값 반환 = 목적은 로스값을 계산하는 것이기 때문에 특별한 결과값을 반환하지 않아도 됨

.. Note::
   **Important detail**: although this module is named ``ContentLoss``, it
   is not a true PyTorch Loss function. If you want to define your content
   loss as a PyTorch Loss function, you have to create a PyTorch autograd function 
   to recompute/implement the gradient manually in the ``backward``
   method.



Style Loss
~~~~~~~~~~

The style loss module is implemented similarly to the content loss
module. It will act as a transparent layer in a
network that computes the style loss of that layer. In order to
calculate the style loss, we need to compute the gram matrix $G_{XL}$. A gram
matrix is the result of multiplying a given matrix by its transposed
matrix. In this application the given matrix is a reshaped version of
the feature maps $F_{XL}$ of a layer $L$. $F_{XL}$ is reshaped to form $\hat{F}_{XL}$, a $K$\ x\ $N$
matrix, where $K$ is the number of feature maps at layer $L$ and $N$ is the
length of any vectorized feature map $F_{XL}^k$. For example, the first line
of $\hat{F}_{XL}$ corresponds to the first vectorized feature map $F_{XL}^1$.

Finally, the gram matrix must be normalized by dividing each element by
the total number of elements in the matrix. This normalization is to
counteract the fact that $\hat{F}_{XL}$ matrices with a large $N$ dimension yield
larger values in the Gram matrix. These larger values will cause the
first layers (before pooling layers) to have a larger impact during the
gradient descent. Style features tend to be in the deeper layers of the
network so this normalization step is crucial.




In [0]:
#스타일 로스를 정의하기 위해 여러 레이어를 사용하는데, 각 레이어들이 가지고 있는 correlation 정보(이미지의 스타일 표현)를 보존하고 표현하기 위해
#Gram matrix(각 레이어들의 출력값을 내적해서 만든 행렬)를 만들어서 레이어들 표현 
def gram_matrix(input):
    a, b, c, d = input.size() #a=배치 크기, b=특징 맵의 크기, c,d=특징 맵의 차원(c*d)

    features = input.view(a * b, c * d)  #F_XL을 \hat F_XL로 사이즈 조정 / view() : 텐서의 모양을 바꾸는 함수

    G = torch.mm(features, features.t())  #torch.mm함수를 이용해 features와 전치시킨 features를 곱함(그램곱)

    return G.div(a * b * c * d) #그램 행렬의 값을 각 특징맵의 요소 숫자로 나누는 방식으로 정규화 수행


Now the style loss module looks almost exactly like the content loss
module. The style distance is also computed using the mean square
error between $G_{XL}$ and $G_{SL}$.




In [0]:
#앞선 콘텐츠 로스 모듈과 동일하게 수행되지만, 타겟과 입력 간의 Gram matrix의 차이를 비교
class StyleLoss(nn.Module):

    def __init__(self, target_feature): 
        super(StyleLoss, self).__init__()
        self.target = gram_matrix(target_feature).detach() #타겟의 특징을 Gram matrix화 + detach()를 사용해 더이상 계산 x
  
    def forward(self, input):
        G = gram_matrix(input)
        self.loss = F.mse_loss(G, self.target) #입력 이미지를 Gram matrix화, 타겟과의 로스 계산
        return input

Importing the Model
-------------------

Now we need to import a pre-trained neural network. We will use a 19
layer VGG network like the one used in the paper.

PyTorch’s implementation of VGG is a module divided into two child
``Sequential`` modules: ``features`` (containing convolution and pooling layers),
and ``classifier`` (containing fully connected layers). We will use the
``features`` module because we need the output of the individual
convolution layers to measure content and style loss. Some layers have
different behavior during training than evaluation, so we must set the
network to evaluation mode using ``.eval()``.




In [0]:
cnn = models.vgg19(pretrained=True).features.to(device).eval() #19개의 레이어 층을 갖는 VGG네트워크를 사전 훈련 네트워크로 사용하기 위해 불러오기

Additionally, VGG networks are trained on images with each channel
normalized by mean=[0.485, 0.456, 0.406] and std=[0.229, 0.224, 0.225].
We will use them to normalize the image before sending it into the network.




In [0]:
#입력 이미지를 신경망으로 보내기 전에 정규화하기 위해 [0.485, 0456, 0.406]의 평균과 [0.229, 0.224, 0.225]의 표준편차로 정의된 모델을 사용
cnn_normalization_mean = torch.tensor([0.485, 0.456, 0.406]).to(device)
cnn_normalization_std = torch.tensor([0.229, 0.224, 0.225]).to(device)

#입력 이미지를 정규화하는 모듈(Normalization)
class Normalization(nn.Module): 
    def __init__(self, mean, std):
        super(Normalization, self).__init__()
        
        #평균과 표준편차 텐서를 [Cx1x1] 형태로 만들어서 입력 이미지 텐서 모양인 [배치 크기 X 채널 값 X 높이 X 넓이]에 연산할 수 있도록 수정
        self.mean = torch.tensor(mean).view(-1, 1, 1) 
        self.std = torch.tensor(std).view(-1, 1, 1)

    def forward(self, img): 
      return (img - self.mean) / self.std #정규화 결과값 반환

A ``Sequential`` module contains an ordered list of child modules. For
instance, ``vgg19.features`` contains a sequence (Conv2d, ReLU, MaxPool2d,
Conv2d, ReLU…) aligned in the right order of depth. We need to add our
content loss and style loss layers immediately after the convolution
layer they are detecting. To do this we must create a new ``Sequential``
module that has content loss and style loss modules correctly inserted.




In [0]:
#스타일/콘텐츠 로스로 계산하고자 하는 레이어들 설정
content_layers_default = ['conv_4'] #콘텐츠 로스 - 'conv_4'
style_layers_default = ['conv_1', 'conv_2', 'conv_3', 'conv_4', 'conv_5'] #스타일 로스 - 'conv_1'~'conv_5'(리스트에 담아둠)

#신경망에 스타일 및 콘텐츠 로스 모듈을 추가하기 위해   
def get_style_model_and_losses(cnn, normalization_mean, normalization_std,
                               style_img, content_img,
                               content_layers=content_layers_default,
                               style_layers=style_layers_default):
  
    cnn = copy.deepcopy(cnn) #원본 유지를 위함

    normalization = Normalization(normalization_mean, normalization_std).to(device) #앞서서 선언한 평균, 표준편차 모델을 정규화한 뒤 normalization 모델에 저장 

    #콘텐츠, 스타일 로스들을 담아 둘 리스트 생성
    content_losses = [] 
    style_losses = []

    model = nn.Sequential(normalization) #앞서 생성한 normalization을 model에 순차적으로 추가

    i = 0 #convolution 레이어를 찾을 때마다 i 증가(카운트변수임)
    for layer in cnn.children(): #cnn의 하위 레이어들에서 레이어 하나를 변수로 반복문 돌림 --> 레이어 하나하나 탐색
        if isinstance(layer, nn.Conv2d): #nn.Conv2d안에 레이어(convl 레이어)가 있으면
            i += 1 #카운트 하나 증가
            name = 'conv_{}'.format(i) #name에 conv_에 카운트 추가한 것을 추가 
        elif isinstance(layer, nn.ReLU): #nn.ReLU안에 레이어가 있으면
            name = 'relu_{}'.format(i) #위와 동일(relu_에 카운트 추가)
            layer = nn.ReLU(inplace=False) #inplace(입력값을 직접 갱신하는 거)는 콘텐츠 로스, 스타일 로스에 좋은 결과를 보여주지 못하기 때문에 False 값을 부여 = out of place로 대체
        elif isinstance(layer, nn.MaxPool2d):#위와 동일(pool_에 카운트 추가)
            name = 'pool_{}'.format(i)
        elif isinstance(layer, nn.BatchNorm2d): #위와 동일(bn_에 카운트 추가)
            name = 'bn_{}'.format(i)
        else: #앞선 항목들에 레이어가 모두 없으면 인식되지 않은 레이어라고 판단
            raise RuntimeError('Unrecognized layer: {}'.format(layer.__class__.__name__))

        model.add_module(name, layer) #model의 레이어에 위에서 찾은 convl 레이어들 추가
        
        #콘텐츠 로스 추가
        if name in content_layers: #콘텐츠 레이어에 위에서 받아온 name이 존재하면
            target = model(content_img).detach() #콘텐츠이미지를 model화 시킨 것의 텐서 추적을 중단하고 타겟으로 설정
            content_loss = ContentLoss(target) #타겟의 콘텐츠 로스 계산
            model.add_module("content_loss_{}".format(i), content_loss) #계산한 콘텐츠 로스들을 모델에 추가
            content_losses.append(content_loss) #content_losses 리스트에 콘텐츠 로스 추가
        
        #스타일 로스 추가 --> 콘텐츠 로스 추가 알고리즘과 동일
        if name in style_layers:
            target_feature = model(style_img).detach()
            style_loss = StyleLoss(target_feature)
            model.add_module("style_loss_{}".format(i), style_loss)
            style_losses.append(style_loss)

    #마지막 콘텐츠/스타일 로스 이후의 레이어들 제외 (1)-->(2)
    for i in range(len(model) - 1, -1, -1): 
        if isinstance(model[i], ContentLoss) or isinstance(model[i], StyleLoss):
            break #(2)만약에 ContentLoss나 StyleLoss 안에 model[i]가 나타난다면 그때 반복 중단

    model = model[:(i + 1)] #(1)model의 i+1번째 요소까지만 저장(반복)

    return model, style_losses, content_losses #model, style_losses, content_losses 반환

Next, we select the input image. You can use a copy of the content image
or white noise.




In [0]:
#입력 이미지 가져오기 --> 콘텐츠 이미지의 복사본이어도 되고, 같은 차원의 백색 노이즈여도 됨. 같은 차원이기만 하면 됨.
input_img = content_img.clone()

plt.figure()
imshow(input_img, title='Input Image')

Gradient Descent
----------------

As Leon Gatys, the author of the algorithm, suggested `here <https://discuss.pytorch.org/t/pytorch-tutorial-for-neural-transfert-of-artistic-style/336/20?u=alexis-jacq>`__, we will use
L-BFGS algorithm to run our gradient descent. Unlike training a network,
we want to train the input image in order to minimise the content/style
losses. We will create a PyTorch L-BFGS optimizer ``optim.LBFGS`` and pass
our image to it as the tensor to optimize.




In [0]:
#콘텐츠/스타일 로스를 최소화하는 방향으로 경사 하강법을 시행해서 학습하기 위해 입력 이미지를 최적화시키고자 함
def get_input_optimizer(input_img):
    optimizer = optim.LBFGS([input_img.requires_grad_()]) #입력 이미지의 경사를 구한 후 최적화시키는 함수 optim.LBFGS()에 넣어서 최적화 실행
    return optimizer #최적화시킨 결과 반환

Finally, we must define a function that performs the neural transfer. For
each iteration of the networks, it is fed an updated input and computes
new losses. We will run the ``backward`` methods of each loss module to
dynamicaly compute their gradients. The optimizer requires a “closure”
function, which reevaluates the modul and returns the loss.

We still have one final constraint to address. The network may try to
optimize the input with values that exceed the 0 to 1 tensor range for
the image. We can address this by correcting the input values to be
between 0 to 1 each time the network is run.




In [0]:
#경사 하강의 반복

def run_style_transfer(cnn, normalization_mean, normalization_std,
                       content_img, style_img, input_img, num_steps=300,
                       style_weight=1000000, content_weight=1): #num_steps의 역할 : 경사 하강의 최대 설정(300)
    """Run the style transfer."""
    print('Building the style transfer model..')
    
    model, style_losses, content_losses = get_style_model_and_losses(cnn,
        normalization_mean, normalization_std, style_img, content_img) #get_style_model_and_losses 함수를 이용해 run_style_transfer에서 사용할 모델, 스타일 및 콘텐츠 로스들을 반환
    
    optimizer = get_input_optimizer(input_img) #입력이미지를 최적화 시킨 후 optimizer에 저장(경사 하강에 사용하기 위함)

    print('Optimizing..')
    run = [0] #경사 하강 횟수 카운트 / 왜 굳이 [0]으로 하는지는 모르겠습니다..ㅜ
    while run[0] <= num_steps: #경사 하강은 300번까지만 가능 

        def closure():
            input_img.data.clamp_(0, 1) #앞에서 최적화된 이미지가 0과 1 사이의 구간에서 머물 수 있도록 .clamp(최소, 최대)함수를 사용해 구간 설정

            optimizer.zero_grad() #경사하강에 들어가기 전, 0으로 초기화
            
            #입력이미지를 신경망에서 사용할 수 있도록 model화시킨 후, 스타일 및 콘텐츠 로스를 계산할 카운트 변수 만들기
            model(input_img)
            style_score = 0
            content_score = 0
            
            #스타일 및 콘텐츠 로스 계산
            for sl in style_losses: #스타일 로스 리스트에서 
                style_score += sl.loss #각 요소들의 로스를 계산한 후 스타일 로스 카운트에 추가
            for cl in content_losses: #콘텐츠 로스 : 위와 동일
                content_score += cl.loss
            
            #계산한 로스들에 스타일 및 콘텐츠 무게를 곱함
            style_score *= style_weight 
            content_score *= content_weight
            
            #최종 로스 = 스타일 + 콘텐츠
            loss = style_score + content_score
            loss.backward() #로스들의 그라디언트 계산

            run[0] += 1 #run 카운트 1 추가
            if run[0] % 50 == 0: #run이 50단위일 때 스타일 및 콘텐츠 로스 출력
                print("run {}:".format(run))
                print('Style Loss : {:4f} Content Loss: {:4f}'.format(
                    style_score.item(), content_score.item()))
                print()

            return style_score + content_score #최종적으로 스타일 로스 및 콘텐츠 로스 스코어 반환

        optimizer.step(closure) #step함수를 통해 지속적으로 optimizer의 closure(위 함수) 결과를 갱신 

    
    input_img.data.clamp_(0, 1) #마지막으로 입력 이미지의 구간 조정 

    return input_img #입력 이미지 반환

Finally, we can run the algorithm.




In [0]:
output = run_style_transfer(cnn, cnn_normalization_mean, cnn_normalization_std,
                            content_img, style_img, input_img) #run_styler_transfer를 통해 입력 이미지의 결과를 output으로 출력

plt.figure() 
imshow(output, title='Output Image')

#이미지 보여주기~!
plt.ioff()
plt.show()


#reference : https://pytorch.org/tutorials/advanced/neural_style_tutorial.html
#코드 해석 참고 : https://pytorch.org/docs/stable/index.html, https://datascienceschool.net/view-notebook/2b5d2a0d176e4982a0c967587b6179e4/